## PART-A

In [1]:
# import wandb
# wandb.login(key = "b4dc866a06ba17317c20de0d13c1a64cc23096dd")
# ENTITY = "cs23s025-indian-institute-of-technology-madras"  
# PROJECT = "CS23S025-Assignment-2-DL"  
# API = wandb.Api()


# runs = API.runs(f"{ENTITY}/{PROJECT}")
# sweep_dict = {}

# for run in runs:
#     if run.sweep:  # Check if the run belongs to a sweep
#         sweep_id = run.sweep.id
#         sweep_name = run.sweep.config.get("name", "Unnamed Sweep")
#         sweep_dict[sweep_id] = sweep_name

# for sweep_id, sweep_name in sweep_dict.items():
#     print(f"Sweep Name: {sweep_name}, Sweep ID: {sweep_id}")



In [2]:
!pip install lightning
import wandb

wandb.login(key = "b4dc866a06ba17317c20de0d13c1a64cc23096dd")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs23s025 (cs23s025-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import torch
from torch import nn, optim
import lightning.pytorch as L


class convNet(L.LightningModule):
    def __init__(
        self,
        img_size,
        activation,
        num_filters,
        filter_size,
        filter_org,
        stride,
        padding,
        dense_neurons,
        learning_rate,
        optimizer,
        dropout,
        usedropout,
        batchnorm,
    ):
        super().__init__()

        # self.img_size = img_size
        self.activation = activation
        self.num_filters = num_filters
        self.kernel_size = filter_size
        self.stride = stride
        self.padding = padding
        self.dense_neurons = dense_neurons
        self.lr = learning_rate
        self.optimizer = optimizer
        self.loss_fn = nn.CrossEntropyLoss()
        self.usedropout = usedropout
        self.batchnorm = batchnorm

        # Define convolutional layers
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=self.num_filters,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn1 = nn.BatchNorm2d(num_features=self.num_filters)

        self.conv2 = nn.Conv2d(
            in_channels=self.num_filters,
            out_channels=self.num_filters * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn2 = nn.BatchNorm2d(num_features=self.conv2.out_channels)

        self.conv3 = nn.Conv2d(
            in_channels=self.conv2.out_channels,
            out_channels=self.conv2.out_channels * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn3 = nn.BatchNorm2d(num_features=self.conv3.out_channels)

        self.conv4 = nn.Conv2d(
            in_channels=self.conv3.out_channels,
            out_channels=self.num_filters * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn4 = nn.BatchNorm2d(num_features=self.conv4.out_channels)

        self.conv5 = nn.Conv2d(
            in_channels=self.conv4.out_channels,
            out_channels=self.num_filters * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn5 = nn.BatchNorm2d(num_features=self.conv5.out_channels)

        # Define activation function based on user input
        if self.activation.lower() == "relu":
            self.activation_layer = nn.ReLU()
        elif self.activation.lower() == "gelu":
            self.activation_layer = nn.GELU()
        elif self.activation.lower() == "silu":
            self.activation_layer = nn.SiLU()
        elif self.activation.lower() == "mish":
            self.activation_layer = nn.Mish()
        else:
            raise ValueError(
                "Invalid activation function. Choose from 'relu', 'gelu', 'mish' or 'silu'"
            )

        # Define max-pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout_layer = nn.Dropout(p=dropout)

        self.flatten = nn.Flatten()

        c1s = int(
            (((img_size - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c2s = int(
            (((c1s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c3s = int(
            (((c2s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c4s = int(
            (((c3s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c5s = int(
            (((c4s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        # print(f"c1s_{c1s}_c2s_{c2s}_c3s_{c3s}_c4s_{c4s}_c5s_{c5s}_out_{self.conv5.out_channels}")

        flatten_neurons = int(c5s * c5s * self.conv5.out_channels)

        # print(f"multi_{c5s*c5s*self.conv5.out_channels}_flat_{flatten_neurons}")

        # Define fully connected layers
        self.fc1 = nn.Linear(flatten_neurons, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, 10)
        self.softmax_layer = nn.Softmax()

    
    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        if self.batchnorm == "Y":
            x = self.bn1(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        x = self.conv2(x)
        if self.batchnorm == "Y":
            x = self.bn2(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        x = self.conv3(x)
        if self.batchnorm == "Y":
            x = self.bn3(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        x = self.conv4(x)
        if self.batchnorm == "Y":
            x = self.bn4(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        x = self.conv5(x)
        if self.batchnorm == "Y":
            x = self.bn5(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        # Flatten
        x = self.flatten(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        # Fully connected layers
        x = self.fc1(x)
        x = self.activation_layer(x)
        x = self.fc2(x)
        # x = self.softmax_layer(x)

        return x

    def training_step(self, batch, batch_idx):
        loss, scores, y, accuracy = self._common_step(batch, batch_idx)

        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy},
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return {"loss": loss, "scores": scores, "y": y}

    def validation_step(self, batch, batch_idx):
        loss, _, _, accuracy = self._common_step(batch, batch_idx)
        self.log_dict(
            {"validation_loss": loss, "validation_accuracy": accuracy},
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y, accuracy = self._common_step(batch, batch_idx)

        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy},
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch

        accuracy = 0

        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        y_pred = torch.argmax(torch.softmax(scores, dim=1), dim=1)
        accuracy += (y_pred == y).sum().item()
        accuracy = accuracy / len(scores)
        return loss, scores, y, accuracy

    def predict_step(self, batch, batch_idx):
        x, _ = batch

        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        if self.optimizer.lower() == "adam":

            return optim.Adam(self.parameters(), lr=self.lr)

        elif self.optimizer.lower() == "sgd":

            return optim.SGD(self.parameters(), lr=self.lr)

        elif self.optimizer.lower() == "nadam":

            return optim.NAdam(self.parameters(), lr=self.lr)

        elif self.optimizer.lower() == "rmsprop":

            return optim.RMSprop(self.parameters(), lr=self.lr)


In [4]:
import lightning.pytorch as L
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


class iNaturalistDataModule(L.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers, img_size, data_augmentation):
        super().__init__()

        self.train_path = data_dir / "train"
        self.test_path = data_dir / "val"
        self.batch_size = batch_size
        self.num_workers = num_workers

        if data_augmentation == "Y":

            self.data_transform = transforms.Compose(
                [
                    transforms.Resize(size=(img_size, img_size)),
                    transforms.AutoAugment(),  # This is the data augmentation method chosen
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.4712, 0.4600, 0.3896], std=[0.2034, 0.1981, 0.1948]
                    ),  # These values are calculated for our dataset
                ]
            )

        elif data_augmentation == "N":

            self.data_transform = transforms.Compose(
                [
                    transforms.Resize(size=(img_size, img_size)),
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.4712, 0.4600, 0.3896], std=[0.2034, 0.1981, 0.1948]
                    ),
                ]
            )

        self.test_data_transform = transforms.Compose(
            [
                transforms.Resize(size=(img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.4748, 0.4645, 0.3965], std=[0.2004, 0.1954, 0.1923]
                ),
            ]
        )

    def setup(self, stage):
        if stage == "fit":

            # First load all the training data
            train_data_full = datasets.ImageFolder(
                root=self.train_path,
                transform=self.data_transform,
                target_transform=None,
            )

            # Decide the number of validation samples required
            validation_samples_per_class = int(0.2 * 1000)

            # These lists will hold the indices of training and validation data samples
            train_indices = []
            val_indices = []

            for class_idx in range(len(train_data_full.classes)):

                # Obtain the indices of each class
                class_indices = [
                    idx
                    for idx, (_, label) in enumerate(train_data_full.imgs)
                    if label == class_idx
                ]

                # Split and add the indices to the respective lists
                val_indices.extend(class_indices[:validation_samples_per_class])
                train_indices.extend(class_indices[validation_samples_per_class:])

            # Create a two subsets of the initially loaded training data as training data and validation data
            self.train_data = Subset(train_data_full, train_indices)
            self.val_data = Subset(train_data_full, val_indices)

        if stage == "test":
            # Load the test data
            self.test_data = datasets.ImageFolder(
                root=self.test_path, transform=self.test_data_transform
            )

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            persistent_workers=True,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            persistent_workers=True,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )


In [5]:
class wandbconfig:

    def set_configs(args):

        wandb_configs = {
            "epochs": args.epochs,
            "img_size": args.img_size,
            "dataug": args.dataug,
            "batchnorm": args.batchnorm,
            "num_filters": args.num_filters,
            "filter_size": args.filter_size,
            "filter_org": args.filter_org,
            "batch_size": args.batch_size,
            "learning_rate": args.learning_rate,
            "dropout": args.dropout,
            "usedropout": args.usedropout,
            "optimizer": args.optimizer,
            "dense_neurons": args.dense_neurons,
            "activation": args.activation,
            "stride": args.stride,
            "padding": args.padding,
        }

        return wandb_configs

    def run_name(wandb_configs):

        run_name = "nf_{}_fsz_{}_fo_{}_a_{}_e_{}_b_{}_dn_{}_da_{}".format(
            wandb_configs["num_filters"],
            wandb_configs["filter_size"],
            wandb_configs["filter_org"],
            wandb_configs["activation"],
            wandb_configs["epochs"],
            wandb_configs["batch_size"],
            wandb_configs["dense_neurons"],
            wandb_configs["dataug"],
        )

        return run_name

    def sweep_name(wandb_configs):

        sweep_name = "nf_{}_fsz_{}_fo_{}_a_{}_e_{}_b_{}_dn_{}".format(
            wandb_configs.num_filters,
            wandb_configs.filter_size,
            wandb_configs.filter_org,
            wandb_configs.activation,
            wandb_configs.epochs,
            wandb_configs.batch_size,
            wandb_configs.dense_neurons,
        )

        return sweep_name

    def tuner_set_configs(args):

        wandb_configs = {
            "epochs": args.epochs,
            "img_size": args.img_size,
            "batch_size": args.batch_size,
            "learning_rate": args.learning_rate,
            "dropout": args.dropout,
        }

        return wandb_configs


In [6]:
import torch
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer
from lightning.fabric.utilities.seed import seed_everything
from pathlib import Path
from lightning.pytorch.callbacks import ModelCheckpoint
#from convolutional import convNet
#from dataset import iNaturalistDataModule
#from wandbConfigs import wandbconfig

device = "gpu" if torch.cuda.is_available() else "cpu"
seed_everything(42)
sweep_configs = {
    "name": "MAX Acc",
    "metric": {"name": "validation_accuracy", "goal": "maximize"},
    "method": "bayes",
    "early_terminate": {"type": "hyperband", "min_iter": 4, "eta": 2},
    "parameters": {
        "img_size": {"values": [256]},
        "num_filters": {"values": [64]},
        "filter_size": {"values": [3]},
        "filter_org": {"values": [2]},
        "activation": {"values": ["gelu"]},
        "optimizer": {"values": ["adam"]},
        "epochs": {"values": [20]},
        "batch_size": {"values": [64]},
        "learning_rate": {"values": [0.001]},
        "dataug": {"values": ["N"]},
        "dense_neurons": {"values": [64]},
        "batchnorm": {"values": ["Y"]},
        "stride": {"values": [1]},
        "padding": {"values": [1]},
        "dropout": {"values": [0.1]},
        "usedropout": {"values": ["Y"]},
    },
}


hyperparameter_defaults = dict()


def train():

    # Initialise wandb

    wandb.init(
        config=hyperparameter_defaults, project="CS23S025-Assignment-2-DL", 
        entity="cs23s025-indian-institute-of-technology-madras"
    )
    wandb_logger = WandbLogger(project="CS23S025-Assignment-2-DL")
    wandb_configs = wandb.config

    sweep_name = wandbconfig.sweep_name(wandb_configs)

    wandb.run.name = sweep_name

    model = convNet(
        img_size=wandb_configs.img_size,
        activation=wandb_configs.activation,
        num_filters=wandb_configs.num_filters,
        filter_size=wandb_configs.filter_size,
        filter_org=wandb_configs.filter_org,
        stride=wandb_configs.stride,
        padding=wandb_configs.padding,
        dense_neurons=wandb_configs.dense_neurons,
        learning_rate=wandb_configs.learning_rate,
        optimizer=wandb_configs.optimizer,
        dropout=wandb_configs.dropout,
        usedropout=wandb_configs.usedropout,
        batchnorm=wandb_configs.batchnorm,
    )

    data = iNaturalistDataModule(
        data_dir=Path("/kaggle/input/naturenew/inaturalist_12K"),
        batch_size=wandb_configs.batch_size,
        num_workers=2,
        img_size=wandb_configs.img_size,
        data_augmentation=wandb_configs.dataug,
    )
    
    #

    checkpoint_callback = ModelCheckpoint(
    monitor="validation_accuracy",
    mode="max",
    save_top_k=1,
    dirpath="/kaggle/working/output/",
    filename="best_model",
    save_weights_only=False  # Save full model
    )
    
    trainer = Trainer(
        accelerator=device,
        callbacks=[checkpoint_callback],
        min_epochs=1,
        max_epochs=wandb_configs.epochs,
        logger=wandb_logger,
        log_every_n_steps=50,
         
    )
    #trainer.fit(model, data)
    trainer.fit(model, datamodule=data)
    trainer.validate(model, data)
    #trainer.test(model, data)
    print("Best model path:", checkpoint_callback.best_model_path)
    best_model_path = checkpoint_callback.best_model_path
   
    wandb.finish()

if __name__ == "__main__":

    sweep_id = wandb.sweep(sweep_configs,
                           entity="cs23s025-indian-institute-of-technology-madras",
                           project="CS23S025-Assignment-2-DL")
    wandb.agent(sweep_id, function=train,count=1)

INFO: Seed set to 42


Create sweep with ID: jjtcz9g6
Sweep URL: https://wandb.ai/cs23s025-indian-institute-of-technology-madras/CS23S025-Assignment-2-DL/sweeps/jjtcz9g6


wandb: Agent Starting Run: 3gvof987 with config:
wandb: 	activation: gelu
wandb: 	batch_size: 64
wandb: 	batchnorm: Y
wandb: 	dataug: N
wandb: 	dense_neurons: 64
wandb: 	dropout: 0.1
wandb: 	epochs: 20
wandb: 	filter_org: 2
wandb: 	filter_size: 3
wandb: 	img_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 64
wandb: 	optimizer: adam
wandb: 	padding: 1
wandb: 	stride: 1
wandb: 	usedropout: Y


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name             | Type             | Params | Mode 
---------------------------------------------------------------
0  | loss_fn          | CrossEntropyLoss | 0      | train
1  | conv1            | Conv2d           | 1.8 K  | train
2  | bn1              | BatchNorm2d      | 128    | train
3  | conv2            | Conv2d           | 73.9 K | train
4  | bn2              | BatchNorm2d      | 256    | train
5  | conv3            | Conv2d           | 295 K  | train
6  | bn3              | BatchNorm2d      | 512    | train
7  | conv4    

...

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    validation_accuracy    │    0.39399999380111694    │
│      validation_loss      │    2.0338504314422607     │
└───────────────────────────┴───────────────────────────┘

Best model path: /kaggle/working/output/best_model.ckpt


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train_accuracy,▁▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇██
train_loss,█▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
validation_accuracy,▁▃▄▄▅▅▆▆▆▆▆▇▇▇███▇▇██
validation_loss,█▆▄▅▃▃▂▂▂▂▃▂▁▃▂▃▂▅▆▆▆
epoch,20
train_accuracy,0.66358
train_loss,0.94918
trainer/global_step,2500
validation_accuracy,0.394


In [2]:
!pip install lightning
import wandb

wandb.login(key = "b4dc866a06ba17317c20de0d13c1a64cc23096dd")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
from torch import nn, optim
import lightning.pytorch as L


class convNet(L.LightningModule):
    def __init__(
        self,
        img_size,
        activation,
        num_filters,
        filter_size,
        filter_org,
        stride,
        padding,
        dense_neurons,
        learning_rate,
        optimizer,
        dropout,
        usedropout,
        batchnorm,
    ):
        super().__init__()

        # self.img_size = img_size
        self.activation = activation
        self.num_filters = num_filters
        self.kernel_size = filter_size
        self.stride = stride
        self.padding = padding
        self.dense_neurons = dense_neurons
        self.lr = learning_rate
        self.optimizer = optimizer
        self.loss_fn = nn.CrossEntropyLoss()
        self.usedropout = usedropout
        self.batchnorm = batchnorm

        # Define convolutional layers
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=self.num_filters,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn1 = nn.BatchNorm2d(num_features=self.num_filters)

        self.conv2 = nn.Conv2d(
            in_channels=self.num_filters,
            out_channels=self.num_filters * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn2 = nn.BatchNorm2d(num_features=self.conv2.out_channels)

        self.conv3 = nn.Conv2d(
            in_channels=self.conv2.out_channels,
            out_channels=self.conv2.out_channels * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn3 = nn.BatchNorm2d(num_features=self.conv3.out_channels)

        self.conv4 = nn.Conv2d(
            in_channels=self.conv3.out_channels,
            out_channels=self.num_filters * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn4 = nn.BatchNorm2d(num_features=self.conv4.out_channels)

        self.conv5 = nn.Conv2d(
            in_channels=self.conv4.out_channels,
            out_channels=self.num_filters * filter_org,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
        )
        self.bn5 = nn.BatchNorm2d(num_features=self.conv5.out_channels)

        # Define activation function based on user input
        if self.activation.lower() == "relu":
            self.activation_layer = nn.ReLU()
        elif self.activation.lower() == "gelu":
            self.activation_layer = nn.GELU()
        elif self.activation.lower() == "silu":
            self.activation_layer = nn.SiLU()
        elif self.activation.lower() == "mish":
            self.activation_layer = nn.Mish()
        else:
            raise ValueError(
                "Invalid activation function. Choose from 'relu', 'gelu', 'mish' or 'silu'"
            )

        # Define max-pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout_layer = nn.Dropout(p=dropout)

        self.flatten = nn.Flatten()

        c1s = int(
            (((img_size - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c2s = int(
            (((c1s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c3s = int(
            (((c2s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c4s = int(
            (((c3s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        c5s = int(
            (((c4s - self.kernel_size + (2 * self.padding)) / self.stride) + 1) / 2
        )

        # print(f"c1s_{c1s}_c2s_{c2s}_c3s_{c3s}_c4s_{c4s}_c5s_{c5s}_out_{self.conv5.out_channels}")

        flatten_neurons = int(c5s * c5s * self.conv5.out_channels)

        # print(f"multi_{c5s*c5s*self.conv5.out_channels}_flat_{flatten_neurons}")

        # Define fully connected layers
        self.fc1 = nn.Linear(flatten_neurons, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, 10)
        self.softmax_layer = nn.Softmax()

    
    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        if self.batchnorm == "Y":
            x = self.bn1(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        x = self.conv2(x)
        if self.batchnorm == "Y":
            x = self.bn2(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        x = self.conv3(x)
        if self.batchnorm == "Y":
            x = self.bn3(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        x = self.conv4(x)
        if self.batchnorm == "Y":
            x = self.bn4(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        x = self.conv5(x)
        if self.batchnorm == "Y":
            x = self.bn5(x)
        x = self.activation_layer(x)
        x = self.pool(x)

        # Flatten
        x = self.flatten(x)

        if self.usedropout == "Y":
            x = self.dropout_layer(x)

        # Fully connected layers
        x = self.fc1(x)
        x = self.activation_layer(x)
        x = self.fc2(x)
        # x = self.softmax_layer(x)

        return x

    def training_step(self, batch, batch_idx):
        loss, scores, y, accuracy = self._common_step(batch, batch_idx)

        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy},
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return {"loss": loss, "scores": scores, "y": y}

    def validation_step(self, batch, batch_idx):
        loss, _, _, accuracy = self._common_step(batch, batch_idx)
        self.log_dict(
            {"validation_loss": loss, "validation_accuracy": accuracy},
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y, accuracy = self._common_step(batch, batch_idx)

        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy},
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch

        accuracy = 0

        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        y_pred = torch.argmax(torch.softmax(scores, dim=1), dim=1)
        accuracy += (y_pred == y).sum().item()
        accuracy = accuracy / len(scores)
        return loss, scores, y, accuracy

    def predict_step(self, batch, batch_idx):
        x, _ = batch

        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        if self.optimizer.lower() == "adam":

            return optim.Adam(self.parameters(), lr=self.lr)

        elif self.optimizer.lower() == "sgd":

            return optim.SGD(self.parameters(), lr=self.lr)

        elif self.optimizer.lower() == "nadam":

            return optim.NAdam(self.parameters(), lr=self.lr)

        elif self.optimizer.lower() == "rmsprop":

            return optim.RMSprop(self.parameters(), lr=self.lr)


In [5]:
import wandb
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go

# Function to log 2D Confusion Matrix
def log_2d_confusion_matrix(cm, class_names):
    fig = ff.create_annotated_heatmap(
        z=cm, 
        x=class_names, 
        y=class_names,
        colorscale = [[0, "lightpink"], [0.5, "darkgoldenrod"], [1, "forestgreen"]],
        showscale=True
    )
    fig.update_layout(
        title=f"2D Confusion Matrix",
        xaxis_title="Predicted Class",
        yaxis_title="Actual Class",
        autosize=True,
        font=dict(size=14)
    )
    wandb.log({"2D Confusion Matrix": wandb.Html(fig.to_html())})

# Function to log 3D Confusion Matrix
def log_3d_confusion_matrix(cm):
    fig_3d = go.Figure(data=[go.Surface(z=cm, 
                                       colorscale= [[0, "pink"], [0.5, "darkgoldenrod"], [1, "green"]])])
    fig_3d.update_layout(
        title="3D Confusion Matrix",
        scene=dict(
            xaxis_title="Predicted Class",
            yaxis_title="Actual Class",
            zaxis_title="Count"
        ),
        autosize=True,
    )
    wandb.log({"3D Confusion Matrix": wandb.Html(fig_3d.to_html())})

# Sweep function to call logging
def sweep_run():
    wandb.init(
        project="CS23S025-Assignment-2-DL",
        entity="cs23s025-indian-institute-of-technology-madras",
        name="predictionTableRun",
        notes="Logging prediction table and confusion matrix",
        tags=["prediction-table", "eval", "val-checkpoint"]
    )

    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load validation dataset
    val_dataset = ImageFolder(root="/kaggle/input/naturenew/inaturalist_12K/val", transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=wandb.config.batch_size, shuffle=False, num_workers=4)

    class_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

    # Load model
    model = convNet.load_from_checkpoint(
        checkpoint_path="/kaggle/input/macacccnn/pytorch/default/1/best_model.ckpt",
        img_size=256,
        activation="gelu",
        num_filters=64,
        filter_size=3,
        filter_org=2,
        stride=1,
        padding=1,
        dense_neurons=64,
        learning_rate=wandb.config.learning_rate,
        optimizer="adam",
        dropout=0.1,
        usedropout="Y",
        batchnorm="Y"
    )

    # Move the model to the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Log prediction table
    def log_prediction_table():
        model.eval()
        columns = ["Image", "Actual Label", "Predicted Label"]
        prediction_table = wandb.Table(columns=columns)
        num_samples = 10
        samples_logged = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                if samples_logged >= num_samples:
                    break

                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                inputs_cpu = inputs.cpu()
                labels_cpu = labels.cpu()
                preds_cpu = preds.cpu()

                for i in range(inputs_cpu.size(0)):
                    if samples_logged >= num_samples:
                        break

                    img_tensor = inputs_cpu[i]
                    true_label = class_names[labels_cpu[i].item()]
                    pred_label = class_names[preds_cpu[i].item()]

                    prediction_table.add_data(
                        wandb.Image(img_tensor),
                        true_label,
                        pred_label
                    )
                    samples_logged += 1

        if samples_logged > 0:
            wandb.log({"Prediction Table": prediction_table})
            print(f"Logged prediction table with {samples_logged} samples.")
        else:
            print("No samples were logged.")

    # Log confusion matrix
    def log_confusion_matrix():
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        cm = confusion_matrix(all_labels, all_preds)

        # Log 2D Confusion Matrix
        log_2d_confusion_matrix(cm, class_names)

        # Log 3D Confusion Matrix
        log_3d_confusion_matrix(cm)

    # Log creative prediction grid
    def log_creative_prediction_grid(model, dataloader, class_names, device, num_images=30):
        model.eval()
        images_shown = 0
        fig, axes = plt.subplots(10, 3, figsize=(12, 30))
        axes = axes.flatten()

        with torch.no_grad():
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                for i in range(inputs.size(0)):
                    if images_shown >= num_images:
                        break

                    img = inputs[i].cpu().permute(1, 2, 0).numpy()
                    img = np.clip(img * 0.229 + 0.485, 0, 1)  # Unnormalize

                    true_label = class_names[labels[i]]
                    pred_label = class_names[preds[i]]

                    correct = pred_label == true_label
                    color = 'green' if correct else 'red'
                    
                    symbol = "[✓]" if correct else "[✗]"
                    title = f"{symbol} {true_label} → {pred_label}"
                    ax = axes[images_shown]
                    ax.imshow(img)
                    ax.set_title(title, color=color, fontsize=10, fontweight='bold')
                    ax.axis('off')

                    images_shown += 1

                if images_shown >= num_images:
                    break

        plt.tight_layout()
        wandb.log({"Creative Prediction Grid": wandb.Image(fig)})
        plt.close()

    # Call the logging functions
    log_prediction_table()
    log_confusion_matrix()
    log_creative_prediction_grid(model, val_loader, class_names, device)
    wandb.finish()

# Sweep configuration with parameters
sweep_config = {
    "name": "prediction-table-sweep-Final",
    "metric": {"name": "validation_accuracy", "goal": "maximize"},
    "method": "bayes",
    "early_terminate": {"type": "hyperband", "min_iter": 4, "eta": 2},
    "parameters": {
        "learning_rate": {
            "min": 1e-5,
            "max": 1e-2,
            "distribution": "uniform"
        },
        "batch_size": {
            "values": [16]
        }
    }
}

# Create the sweep
sweep_id = wandb.sweep(sweep_config, entity="cs23s025-indian-institute-of-technology-madras", project="CS23S025-Assignment-2-DL")

# Run the sweep agent
wandb.agent(sweep_id, function=sweep_run, count=1)


Create sweep with ID: cako0b50
Sweep URL: https://wandb.ai/cs23s025-indian-institute-of-technology-madras/CS23S025-Assignment-2-DL/sweeps/cako0b50


wandb: Agent Starting Run: j2ere8sw with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.00394508348548209


Logged prediction table with 10 samples.


## PART - B 

In [ ]:
class wandbconfig:

    def set_configs(args):

        wandb_configs = {
            "epochs": args.epochs,
            "img_size": args.img_size,
            "dataug": args.dataug,
            "batchnorm": args.batchnorm,
            "num_filters": args.num_filters,
            "filter_size": args.filter_size,
            "filter_org": args.filter_org,
            "batch_size": args.batch_size,
            "learning_rate": args.learning_rate,
            "dropout": args.dropout,
            "usedropout": args.usedropout,
            "optimizer": args.optimizer,
            "dense_neurons": args.dense_neurons,
            "activation": args.activation,
            "stride": args.stride,
            "padding": args.padding,
        }

        return wandb_configs

    def run_name(wandb_configs):

        run_name = "nf_{}_fsz_{}_fo_{}_a_{}_e_{}_b_{}_dn_{}_da_{}".format(
            wandb_configs["num_filters"],
            wandb_configs["filter_size"],
            wandb_configs["filter_org"],
            wandb_configs["activation"],
            wandb_configs["epochs"],
            wandb_configs["batch_size"],
            wandb_configs["dense_neurons"],
            wandb_configs["dataug"],
        )

        return run_name

    def sweep_name(wandb_configs):

        sweep_name = "nf_{}_fsz_{}_fo_{}_a_{}_e_{}_b_{}_dn_{}".format(
            wandb_configs.num_filters,
            wandb_configs.filter_size,
            wandb_configs.filter_org,
            wandb_configs.activation,
            wandb_configs.epochs,
            wandb_configs.batch_size,
            wandb_configs.dense_neurons,
        )

        return sweep_name

    def tuner_set_configs(args):

        wandb_configs = {
            "epochs": args.epochs,
            "img_size": args.img_size,
            "batch_size": args.batch_size,
            "learning_rate": args.learning_rate,
            "dropout": args.dropout,
        }

        return wandb_configs


In [ ]:
import lightning.pytorch as L
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


class iNaturalistDataModule(L.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers, img_size, data_augmentation):
        super().__init__()

        self.train_path = data_dir /"train"
        self.test_path = data_dir /"val"
        self.batch_size = batch_size
        self.num_workers = num_workers

        if data_augmentation == "Y":

            self.data_transform = transforms.Compose(
                [
                    transforms.Resize(size=(img_size, img_size)),
                    transforms.AutoAugment(),  # This is the data augmentation method chosen
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.4712, 0.4600, 0.3896], std=[0.2034, 0.1981, 0.1948]
                    ),  # These values are calculated for our dataset
                ]
            )

        elif data_augmentation == "N":

            self.data_transform = transforms.Compose(
                [
                    transforms.Resize(size=(img_size, img_size)),
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.4712, 0.4600, 0.3896], std=[0.2034, 0.1981, 0.1948]
                    ),
                ]
            )

        self.test_data_transform = transforms.Compose(
            [
                transforms.Resize(size=(img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.4748, 0.4645, 0.3965], std=[0.2004, 0.1954, 0.1923]
                ),
            ]
        )

    def setup(self, stage):
        if stage == "fit":

            # First load all the training data
            train_data_full = datasets.ImageFolder(
                root=self.train_path,
                transform=self.data_transform,
                target_transform=None,
            )

            # Decide the number of validation samples required
            validation_samples_per_class = int(0.2 * 1000)

            # These lists will hold the indices of training and validation data samples
            train_indices = []
            val_indices = []

            for class_idx in range(len(train_data_full.classes)):

                # Obtain the indices of each class
                class_indices = [
                    idx
                    for idx, (_, label) in enumerate(train_data_full.imgs)
                    if label == class_idx
                ]

                # Split and add the indices to the respective lists
                val_indices.extend(class_indices[:validation_samples_per_class])
                train_indices.extend(class_indices[validation_samples_per_class:])

            # Create a two subsets of the initially loaded training data as training data and validation data
            self.train_data = Subset(train_data_full, train_indices)
            self.val_data = Subset(train_data_full, val_indices)

        if stage == "test":
            # Load the test data
            self.test_data = datasets.ImageFolder(
                root=self.test_path, transform=self.test_data_transform
            )

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            persistent_workers=True,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            persistent_workers=True,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )


In [ ]:
import torch
from torch import nn, optim
import lightning.pytorch as L
import torchvision.models

class efficientNet(L.LightningModule):

    def __init__(self, dropout, learning_rate):
        super().__init__()

        self.dropout = dropout
        self.learning_rate = learning_rate
        
        weights = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT #We obtain the best available weights for this model
        model = torchvision.models.efficientnet_v2_s(weights=weights) #Initializing the model

        #We use the approach of overwriting the classifier layer, so that output classes is 10.
        model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=dropout, inplace=True), 
        torch.nn.Linear(in_features=1280, 
                        out_features=10, #Here initially it was 1000
                        bias=True))

        #We are freezing all the features layers. This does not include freezing of the classifier layer.
        for param in model.features.parameters():
            param.requires_grad = False

        self.efNet_tuned = model

        self.loss_fn = nn.CrossEntropyLoss()

    
    def forward(self, x):
        x = self.efNet_tuned(x)
        return x


    def training_step(self, batch, batch_idx):
        loss, scores, y, accuracy = self._common_step(batch, batch_idx)
 
        self.log_dict(
            {
                "train_loss": loss,
                "train_accuracy": accuracy
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return {"loss": loss, "scores": scores, "y": y}

    def validation_step(self, batch, batch_idx):
        loss, scores, y, accuracy = self._common_step(batch, batch_idx)
        self.log_dict(
            {
                "validation_loss": loss,
                "validation_accuracy": accuracy
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def test_step(self, batch, batch_idx):
        loss, _, _, accuracy = self._common_step(batch, batch_idx)
        self.log_dict(
            {
                "test_loss": loss,
                "test_accuracy": accuracy
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch
        
        accuracy = 0
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        y_pred = torch.argmax(torch.softmax(scores, dim=1), dim=1)
        accuracy += (y_pred == y).sum().item()
        accuracy = accuracy/len(scores)
        return loss, scores, y, accuracy

    def predict_step(self, batch, batch_idx):
        x, _ = batch
        
        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
         return optim.Adam(self.parameters(), lr=self.learning_rate)

In [ ]:
import torch
import argparse
import wandb
from types import SimpleNamespace
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer
from lightning.fabric.utilities.seed import seed_everything
from pathlib import Path

# from efficientNet_v2 import efficientNet
# from dataset import iNaturalistDataModule
# from wandbConfigs import wandbconfig

device = "gpu" if torch.cuda.is_available() else "cpu"


seed_everything(42)


def train(wandb_configs, args):

    # Initialise wandb
    run_name = "EfficientNetV2_Freeze_All-Final"
    wandb.init(
        config=wandb_configs,
        project=args.wandb_project,
        entity=args.wandb_entity,
        name=run_name,
    )
    wandb_logger = WandbLogger(project=args.wandb_project)
    wandb_configs = wandb.config

    model = efficientNet(0.2, 0.001)

    data = iNaturalistDataModule(
        data_dir=Path(args.path),
        batch_size=wandb_configs.batch_size,
        num_workers=2,
        img_size=wandb_configs.img_size,
        data_augmentation="N",
    )
    trainer = Trainer(
        accelerator=device,
        min_epochs=1,
        max_epochs=wandb_configs.epochs,
        logger=wandb_logger,
        log_every_n_steps=50,
    )

    trainer.fit(model, data)
    trainer.validate(model, data)
    # trainer.test(model, data)

    wandb.finish()


if __name__ == "__main__":

    args = SimpleNamespace(
    wandb_project="CS23S025-Assignment-2-DL",
    wandb_entity="cs23s025-indian-institute-of-technology-madras",
    path="/kaggle/input/naturenew/inaturalist_12K",
    epochs=15,
    img_size=224,
    batch_size=64,
    learning_rate=0.001,
    dropout=0.1,
    )

    # Get wandb configs
    wandb_configs = wandbconfig.tuner_set_configs(args)
    
    # Run training
    train(wandb_configs, args)
